In [ ]:
# Grab the return_num and num_returns dimensions

def gp(las):

        ground_points = (las.number_of_returns == las.return_number)

        print("%i points out of %i were ground points." % (len(ground_points),
                len(las.points)))

        return ground_points

def hist(par):

        fig = plt.figure(figsize=(10,6))
        plt.hist(par, bins=50)
        plt.title("Histogram of the Intensity Dimension")
        plt.show()

## Get the foliage

## Approach 1: percentiles

In [ ]:
# select the field
keep = (df['y'] > 0.34*df['x'] - 4.0) & (df['y'] < 0.34*df['x'] + 6)
keep &= (df['x'] < -0.35*df['y'] - 10) & (df['x'] > -0.35*df['y'] - 60)

# Set the foliage min and max height
keepz = (df['z'] > 1.2) & (df['z'] < 3.8)

# remove outliers using percentiles
res, mask = loads.remove_outliers(df['x'][keep & keepz], df['z'][keep & keepz])
leaves = np.zeros(len(df['x']), dtype=bool)
leaves[(keep) & (keepz)] = mask

In [ ]:
keep = PCsample
df1 = df[keep][::1]
print(len(df))
print(len(df1))

plt.figure(figsize=(16, 8))
plt.scatter(df1['x'], df1['z'], s=0.01)
plt.axhline(1.2, lw=2, c='k')
plt.axhline(3.8, lw=2, c='k')

res, _ = loads.remove_outliers(df['x'][keep & keepz][::5], df['z'][keep & keepz][::5])

plt.grid()

plt.ylim(-2,10)

plt.xlabel(r'$x$', size=20)
plt.ylabel(r'$z$', size=20)

### Colour mask: RGB clustering with K-means

In [ ]:
# keepz = (las.z > 1.2) & (las.z < 3.5)
keep = PCsample #& keepz

Y = colors[keep]/2**8
N = 8
clt = KMeans(n_clusters = N, init='k-means++')
clt.fit(Y)

hist = figures.centroid_histogram(clt)
bar = figures.plot_colors(hist, clt.cluster_centers_)
# maxbar = plot_max()

p = plt.figure()
p2 = p.add_subplot(111)
p2.axis("off")
p2.imshow(bar)

In [ ]:
labels = clt.labels_

keeplab = np.zeros(len(points[keep]), dtype=bool)
for i in [1]:
    keeplab |= (labels == i)

pointslist = [points[keep][~keeplab]]
colours = [colors[keep][~keeplab]/2**16]
loads.showPCDS(pointslist, colours)

### Intensity mask: Canes removal with low intensity values

In [ ]:
# intensity = np.sqrt(las.intensity[keep])
# intensity = (las.intensity[keep])**5

keep = PCsample
# intensity = np.sin(las.red[keep]/2**16 + las.green[keep]/2**16 + las.blue[keep]/2**16)**2
intensity = (las.intensity[keep])**0.5
N = 5

tot = len(intensity)
ini, fin = intensity.min(), intensity.max()
bins = np.linspace(ini, fin, N+1)
masks = []

for i in range(len(bins[:-1])):

    mask = np.logical_and(intensity >= bins[i], intensity < bins[i+1])
    masks.append(mask)

    print(i, bins[i], bins[i+1], np.round(100*mask.sum()/tot, 2))


# plot the two point clouds
coltmp = plt.cm.jet(np.linspace(0,1,N))[:,0:3]

maskstmp = masks[0:1]
coltmp2 = coltmp[0:1]

pointslist = [points[keep][i] for i in maskstmp]
colours = [list(i) for i in coltmp2]
loads.showPCDS(pointslist, colours)


### Geometrical mask: poles removal with circular masks in x and y coordinates at certain heigh in z

In [ ]:
keepz = (las.z > 4) & (las.z < 4.5)
keep = PCsample & keepz

Y = points[keep]
N = 10
clt = KMeans(n_clusters = N, init='k-means++')
clt.fit(Y)

In [ ]:
clt.labels_

In [ ]:

keeplab = {} #np.zeros(len(clusters), dtype=bool)
for i in set(clusters):
    keeplab[str(i)] = (clusters == i)

# pointslist = [points[PCsample][::10][keeplab]]
# colours = [colors[PCsample][::10][keeplab]/2**16]
# loads.showPCDS(pointslist, colours)

N = len(set(clusters))
coltmp = plt.cm.jet(np.linspace(0,1,N))[:,0:3]

# maskstmp = masks[0:1]
# coltmp2 = coltmp[0:1]

pointslist = [points[PCsample][::10][i] for i in keeplab.values()]
colours = [list(i) for i in coltmp]
loads.showPCDS(pointslist, colours)

In [ ]:
leaves = (labels != 1) & (~np.logical_and(intensity >= 0, intensity < 2.1))

In [ ]:
keep = PCsample
df1 = df[keep][leaves]
print(len(df[keep]))
print(len(df1))

plt.figure(figsize=(16, 8))

plt.scatter(df1['x'], df1['z'], s=0.01)
plt.axhline(1.3, lw=2, c='k')
plt.axhline(3.2, lw=2, c='k')
plt.axhline(2.5, lw=2, ls='--', c='k')

for i in [-22.75, -17.5, -14.25, -12.35]:
    plt.axvline(i, c='b')

# for i in [1.1, -4.6, -6.6, -8.3]:
#     plt.axvline(i, c='b')

# plt.grid()

plt.ylim(-2,10)
zzz
plt.xlabel(r'$x$', size=20)
plt.ylabel(r'$z$', size=20)

In [ ]:
slide = (np.array(las.y) > 0.34*np.array(las.x) - 0.5 - 0.3) & (np.array(las.y) < 0.34*np.array(las.x) - 0.5 + 0.3)
keep = PCsample & slide & (las.z > 0.3) & (las.z < 1.8)
Ntot = keep.sum()
print(Ntot)

db = DBSCAN(eps=0.2).fit(X[keep])
# N = 30
# db = KMeans(init='k-means++', n_clusters=N).fit(X[keep])

In [ ]:
labels = {}

for i in set(db.labels_):
    mask = db.labels_ == i
    perc = 100*mask.sum()/Ntot
    if perc > 0:
        print(i, perc)
        labels[i] = mask

N = len(list(labels.keys()))

# plot the two point clouds
coltmp = plt.cm.jet(np.linspace(0,1,N))[:,0:3]

pointslist = [points[keep][val] for val in labels.values()]
colours = [list(i) for i in coltmp]
loads.showPCDS(pointslist, colours)

In [ ]:
N = 30
keep = PCsample & (las.z > 2.5) & (las.z < 4.5)
Ntot = keep.sum()

kmeans = KMeans(init='k-means++',
        n_clusters=N,
        # n_init=10,
        # max_iter=500,
        # random_state=42
        ).fit(points[keep])

labels = {}

for i in set(kmeans.labels_):
    labels[i] = kmeans.labels_ == i
    print(i, 100*labels[i].sum()/Ntot)

# plot the two point clouds
coltmp = plt.cm.jet(np.linspace(0,1,N))[:,0:3]

pointslist = [points[keep][val] for val in labels.values()]
colours = [list(i) for i in coltmp]
loads.showPCDS(pointslist, colours)